In [16]:
import pickle
import numpy as np
import pandas as pd
import ollama

In [17]:
df = pd.read_csv(r"C:\Users\223099055.HCAD\Downloads\DANA\dana_transactions.csv")

# Generate merchant gender using LLM

In [3]:
def get_gender_from_merchant(merchant_name):
    prompt = f"What gender does {merchant_name} store sell products for? Say don't know if you have no idea"
    response = ollama.chat(model='tinyllama:latest', messages=[
      {'role': 'user', 'content': prompt,},
    ])
    gender = "all"
    res = response['message']['content'].lower()
    if " men " in res and " women " in res:
        pass
    elif " men " in res:
        gender = "men"
    elif " women " in res:
        gender = "women"
    return gender

In [4]:
merchant_to_gender = {}

with open('merchant_to_gender.pickle', 'rb') as f:
    merchant_to_gender = pickle.load(f)

In [10]:
for i in df['merchant_id'].unique():
    if i in merchant_to_gender:
        continue
    d = df[df['merchant_id'] == i].iloc[0]
    g = get_gender_from_merchant(d['merchant_name'])
    merchant_to_gender[i] = g

In [12]:
with open('merchant_to_gender.pickle', 'wb') as f:
    pickle.dump(merchant_to_gender, f, protocol=pickle.HIGHEST_PROTOCOL)

# Generate gender rate for each merchant category

In [21]:
merchant_to_gender = {}

with open('merchant_to_gender.pickle', 'rb') as f:
    merchant_to_gender = pickle.load(f)

In [22]:
merchant_cat_gender_rate = {} 

for c in df['merchant_category_id'].unique():
    df_c = df[df['merchant_category_id'] == c].copy()
    merc_cat_gender = []
    for m in df_c['merchant_id'].unique():
        if m in merchant_to_gender:
            merc_cat_gender.append(merchant_to_gender[m])

    men_rate = merc_cat_gender.count("men") / len(merc_cat_gender)
    women_rate = merc_cat_gender.count("women") / len(merc_cat_gender)
    
    merchant_cat_gender_rate[c] = {
        'men': men_rate,
        'women': women_rate,
    }

# Generate user gender

In [103]:
user_gender = {}

for u in df['user_id'].unique():
    df_u = df[df['user_id'] == u].copy()
    user_gen = None

    merch_genders = []
    for m in df_u['merchant_id'].unique():
        if m in merchant_to_gender:
            merch_genders.append(merchant_to_gender[m])

    if 'men' not in merch_genders and 'women' not in merch_genders:
        continue

    if merch_genders.count("men") > merch_genders.count("women"):
        user_gen = 'male'
    else:
        user_gen = 'female'

    if user_gen:
        user_gender[u] = user_gen

In [98]:
for u in df['user_id'].unique():
    if u in user_gender:
        continue

    df_u = df[df['user_id'] == u].copy()
    merch_genders = []
    user_gen = None

    for c in df_u['merchant_category_id'].unique():
        rate = merchant_cat_gender_rate[c]
        rate_unk = 1 - (rate['men'] + rate['women'])
        option = ['men', 'women', 'unk']
        prob = [rate['men'], rate['women'], rate_unk]
        g = np.random.choice(option, 1, p=prob)
        merch_genders.append(str(g[0]))
    
    if 'men' not in merch_genders and 'women' not in merch_genders:
        continue
    
    if merch_genders.count("men") > merch_genders.count("women"):
        user_gen = 'male'
    else:
        user_gen = 'female'
    
    if user_gen:
        user_gender[u] = user_gen

In [100]:
with open('user_gender.pickle', 'wb') as f:
    pickle.dump(user_gender, f, protocol=pickle.HIGHEST_PROTOCOL)

In [99]:
len(user_gender)

24097

In [96]:
len(df['user_id'].unique())

24321

In [101]:
24321 - 24097

224

# Debug

In [ ]:
merchant_name = "Barron, Davis and Parks"
prompt = f"What gender does {merchant_name} store sell products for? Say don't know if you have no idea. Please pick from these options: men, women, both"
response = ollama.chat(model='tinyllama:latest', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])
print(response['message']['content'])